In [10]:
import cv2
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from PIL import Image
import PIL.ImageOps
import os,ssl,time
import time

In [5]:
y = pd.read_csv("C:/Users/ADMIN/Desktop/WhiteHat_Jr/General/Data/Alphabet-Recognition-1.csv")["labels"]
X = np.load("C:/Users/ADMIN/Desktop/WhiteHat_Jr/General/Data/Alphabet-Recognition-Image-1.npz")["arr_0"]

In [6]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=9,test_size=0.25)

X_train_scaled = X_train/255
X_test_scaled = X_test/255  

In [7]:
clf = LogisticRegression(solver='saga', multi_class='multinomial').fit(X_train_scaled,y_train)

c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [9]:
y_pred = clf.predict(X_test_scaled)
accuracy = accuracy_score(y_test,y_pred)
print("Accuracy : ", accuracy)

Accuracy :  1.0


In [20]:
cap = cv2.VideoCapture(0)

t_end = time.time() + 25
while time.time() < t_end:
    try:
        ret,frame = cap.read()
        gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

        height, width = gray.shape()
        upper_left = (int(width / 2 - 56), int(height / 2 - 56))
        bottom_right = (int(width / 2 + 56), int(height / 2 + 56))
        cv2.rectangle(gray, upper_left, bottom_right, (0, 255, 0), 2)

        roi = gray[upper_left[1]:bottom_right[1], upper_left[0]:bottom_right[0]]

        im_pil =   Image.fromarray(roi)
        image_bg = im_pil.convert("L")

        image_bg_resized = image_bg.resize((28,28),Image.ANTIALIAS)

        image_bg_resized_inverted = PIL.ImageOps.invert(image_bg_resized)
        
        max_filter = 20
        min_pixel = np.percentile(image_bg_resized_inverted,max_filter)
        image_bg_resized_inverted_scaled = np.clip(image_bg_resized_inverted - min_pixel, 0, 255)

        max_pixel = np.max(image_bg_resized_inverted)
        image_bg_resized_inverted_scaled = np.asarray(image_bg_resized_inverted_scaled)/max_pixel

        test_sample = np.array(image_bg_resized_inverted_scaled).reshape(1,174)
        test_pred = clf.predict(test_sample)
        print("Prediction : ", test_pred)
        cv2.imshow("frame",gray)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    except Exception as e:
        pass


cap.release()
cv2.destroyAllWindows()